In [2]:
#importacion de libraries
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
#carga de datasets
#Le especifico el tipo a algunas columnas porque sino me tira un warning, podria especificar el verdadero tipo tambien
todoSet = pd.read_csv("/home/jonathan/Escritorio/orgaDeDatos/fiuba-trocafone-tp2-final-set/events_up_to_01062018.csv",\
                   dtype = {3: object,5: object,
                    6: object,7: object,
                    8: object,9: object,
                    10: object,11: object,
                    12: object,13: object,
                    14: object,15: object,
                    16: object,17: object,
                    18: object,19: object,
                    20: object,21: object,
                    22: object})
trainSet = pd.read_csv("/home/jonathan/Escritorio/orgaDeDatos/fiuba-trocafone-tp2-final-set/labels_training_set.csv")
#usersJunioSet = pd.read_csv("/home/jonathan/Escritorio/orgaDeDatos/fiuba-trocafone-tp2-final-set/trocafone_kaggle_test.csv")

In [3]:
todoSet.head()

,timestamp,event,person,url,sku,model,condition,storage,color,skus,...,search_engine,channel,new_vs_returning,city,region,country,device_type,screen_resolution,operating_system_version,browser_version
0,2018-05-18 00:11:59,viewed product,4886f805,NaN,9288.0,Samsung Galaxy J7 Prime,Excelente,32GB,Dourado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-05-18 00:11:27,viewed product,ad93850f,NaN,304.0,iPhone 5s,Muito Bom,32GB,Cinza espacial,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2018-05-18 00:11:16,viewed product,0297fc1e,NaN,6888.0,iPhone 6S,Muito Bom,64GB,Prateado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2018-05-18 00:11:14,viewed product,2d681dd8,NaN,11890.0,iPhone 7,Bom,128GB,Vermelho,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2018-05-18 00:11:09,viewed product,cccea85e,NaN,7517.0,LG G4 H818P,Excelente,32GB,Branco,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
trainSet.head()

,person,label
0,0566e9c1,0
1,6ec7ee77,0
2,abe7a2fb,0
3,34728364,0
4,87ed62de,0


In [5]:
#usersJunioSet.head()

,person
0,4886f805
1,0297fc1e
2,2d681dd8
3,cccea85e
4,4c8a8b93


In [6]:
todoSet.dtypes

timestamp                    object
event                        object
person                       object
url                          object
sku                         float64
model                        object
condition                    object
storage                      object
color                        object
skus                         object
search_term                  object
staticpage                   object
campaign_source              object
search_engine                object
channel                      object
new_vs_returning             object
city                         object
region                       object
country                      object
device_type                  object
screen_resolution            object
operating_system_version     object
browser_version              object
dtype: object

In [6]:
#conversion de Object a datetime
todoSet['timestamp'] = pd.to_datetime(todoSet['timestamp'])

# Feature Engineering

In [7]:
#Comienzo buscando features sobre la cantidad de eventos por usuario

In [8]:
#Me quedo con los usuarios que convirtieron en junio(los de train)
usersTrain = pd.merge(todoSet,trainSet,how = 'inner',on='person')

In [9]:
#Creo una columna con solo fecha sin tiempo
usersTrain['fecha'] = usersTrain['timestamp'].dt.date

### Features numericos

In [10]:
usersTrain['event'].value_counts()

viewed product       629233
brand listing        105993
visited site         102299
ad campaign hit       94998
generic listing       80453
searched products     63887
search engine hit     52829
checkout              32539
staticpage             5660
conversion             3564
lead                    431
Name: event, dtype: int64

In [11]:
def agruparPorPersonaFecha(dataframe,evento,nuevoNombreEvento):
    feature = dataframe[dataframe['event'] == evento].sort_values(by = 'fecha')
    feature = dataframe.groupby(['fecha','person'])['event'].size()
    feature = pd.DataFrame(data = feature)\
                        .rename(columns = {'event' : nuevoNombreEvento})
    feature = feature.sort_values(by = ['fecha','person']).reset_index()
    feature['fecha'] = pd.to_datetime(feature['fecha'])
    feature['mes'] = feature['fecha'].dt.month
    feature = feature[['mes','person',nuevoNombreEvento]]
    feature = pd.DataFrame(data = feature.groupby(['mes','person'])[nuevoNombreEvento]\
                             .size()).unstack('mes',0).reset_index('person')
    feature = pd.merge(feature,trainSet,on='person',how='inner')\
                             .drop(('person', ''),1).rename(columns = {\
                                (nuevoNombreEvento, 1) : 'Enero - ' + nuevoNombreEvento,\
                                (nuevoNombreEvento, 2) : 'Febrero - ' + nuevoNombreEvento,\
                                 (nuevoNombreEvento, 3) : 'Marzo - ' + nuevoNombreEvento,\
                                 (nuevoNombreEvento, 4) : 'Abril - ' + nuevoNombreEvento,\
                                (nuevoNombreEvento, 5) : 'Mayo - ' + nuevoNombreEvento})
    return feature

In [12]:
# 1er feature: Cantidad de eventos por usuario
eventosPorUserTrain = usersTrain.groupby(['fecha','person'])['event'].size()
eventosPorUserTrain = pd.DataFrame(data = eventosPorUserTrain)\
                        .rename(columns = {'event' : 'cantEventos'})
eventosPorFechaTrain = eventosPorUserTrain.sort_values(by = ['fecha','person']).reset_index()
eventosPorFechaTrain['fecha'] = pd.to_datetime(eventosPorFechaTrain['fecha'])
eventosPorFechaTrain['mes'] = eventosPorFechaTrain['fecha'].dt.month
eventosPorFechaTrain = eventosPorFechaTrain[['mes','person','cantEventos']]
eventosPorFechaTrain = pd.DataFrame(data = eventosPorFechaTrain.groupby(['mes','person'])['cantEventos']\
                             .size()).unstack('mes',0).reset_index('person')
eventosPorFechaTrain = pd.merge(eventosPorFechaTrain,trainSet,on='person',how='inner').drop(('person', ''),1)\
    .rename(columns = {\
                        ('cantEventos', 1) : 'Enero - cantidadEventos',\
                        ('cantEventos', 2) : 'Febrero - cantidadEventos',\
                         ('cantEventos', 3) : 'Marzo - cantidadEventos',\
                         ('cantEventos', 4) : 'Abril - cantidadEventos',\
                        ('cantEventos', 5) : 'Mayo - cantidadEventos'\
})
eventosPorFechaTrain.head()

/home/jonathan/anaconda3/lib/python3.6/site-packages/pandas/core/reshape/merge.py:543: UserWarning: merging between different levels can give an unintended result (2 levels on the left, 1 on the right)
  warnings.warn(msg, UserWarning)


,person,Enero - cantidadEventos,Febrero - cantidadEventos,Marzo - cantidadEventos,Abril - cantidadEventos,Mayo - cantidadEventos,label
0,0008ed71,0,0,0,0,1,0
1,000c79fe,0,0,0,0,1,0
2,001802e4,0,0,0,0,1,0
3,0019e639,1,3,0,1,9,0
4,001b0bf9,0,0,0,0,1,0


In [13]:
#2do feature: cantidad de checkouts por user por dia 

usersChecks = agruparPorPersonaFecha(usersTrain,'checkout','checkouts')


/home/jonathan/anaconda3/lib/python3.6/site-packages/pandas/core/reshape/merge.py:543: UserWarning: merging between different levels can give an unintended result (2 levels on the left, 1 on the right)
  warnings.warn(msg, UserWarning)


In [14]:
#3er feature: cantidad de leads por mes por user
usersLeads = agruparPorPersonaFecha(usersTrain,'lead','leads')


/home/jonathan/anaconda3/lib/python3.6/site-packages/pandas/core/reshape/merge.py:543: UserWarning: merging between different levels can give an unintended result (2 levels on the left, 1 on the right)
  warnings.warn(msg, UserWarning)


In [15]:
#4to feature: cantidad de conversiones por mes por user
usersConversions = agruparPorPersonaFecha(usersTrain,'conversion','conversiones')


/home/jonathan/anaconda3/lib/python3.6/site-packages/pandas/core/reshape/merge.py:543: UserWarning: merging between different levels can give an unintended result (2 levels on the left, 1 on the right)
  warnings.warn(msg, UserWarning)


In [16]:
#5to feature: cantidad de staticPages visitadas por dia por user
usersStaticPages = agruparPorPersonaFecha(usersTrain,'staticpage','staticPageVisitadas')


/home/jonathan/anaconda3/lib/python3.6/site-packages/pandas/core/reshape/merge.py:543: UserWarning: merging between different levels can give an unintended result (2 levels on the left, 1 on the right)
  warnings.warn(msg, UserWarning)


In [17]:
#6to feature: cantidad de viewed product por mes por usuario
usersViewedProducts = agruparPorPersonaFecha(usersTrain,'viewed product','ProductosVistosPorUser')


/home/jonathan/anaconda3/lib/python3.6/site-packages/pandas/core/reshape/merge.py:543: UserWarning: merging between different levels can give an unintended result (2 levels on the left, 1 on the right)
  warnings.warn(msg, UserWarning)


In [18]:
#7mo feature: cantidad de listados visitados por usuario por mes
usersBrandListing = agruparPorPersonaFecha(usersTrain,'brand listing','listadosVisitados')

/home/jonathan/anaconda3/lib/python3.6/site-packages/pandas/core/reshape/merge.py:543: UserWarning: merging between different levels can give an unintended result (2 levels on the left, 1 on the right)
  warnings.warn(msg, UserWarning)


In [19]:
#8vo feature: numero de recurrencias por usuario
usersRecurrencias = agruparPorPersonaFecha(usersTrain,'new_vs_returning','recurrencias')

/home/jonathan/anaconda3/lib/python3.6/site-packages/pandas/core/reshape/merge.py:543: UserWarning: merging between different levels can give an unintended result (2 levels on the left, 1 on the right)
  warnings.warn(msg, UserWarning)


In [20]:
#9no feature: cantidad de searched products
usersSearchProduct = agruparPorPersonaFecha(usersTrain,'searched products','productosBuscados')

/home/jonathan/anaconda3/lib/python3.6/site-packages/pandas/core/reshape/merge.py:543: UserWarning: merging between different levels can give an unintended result (2 levels on the left, 1 on the right)
  warnings.warn(msg, UserWarning)


In [21]:
#10mo feature: cantidad de veces que el usuario ingresa al sitio mediante campania de marketing online
usersCampaignMarketing = agruparPorPersonaFecha(usersTrain,'ad campaign hit','accesos por marketing online')


/home/jonathan/anaconda3/lib/python3.6/site-packages/pandas/core/reshape/merge.py:543: UserWarning: merging between different levels can give an unintended result (2 levels on the left, 1 on the right)
  warnings.warn(msg, UserWarning)


In [22]:
#11 feature: cantidad de veces que el user ingresa al sitio mediante search engine
usersSearchEngine =  agruparPorPersonaFecha(usersTrain,'search engine hit','searchEngine')


/home/jonathan/anaconda3/lib/python3.6/site-packages/pandas/core/reshape/merge.py:543: UserWarning: merging between different levels can give an unintended result (2 levels on the left, 1 on the right)
  warnings.warn(msg, UserWarning)


In [23]:
#12 feature: cantidad de veces que el usuario visita la homepage
usersHomePage = agruparPorPersonaFecha(usersTrain,'generic listing','accesos a la homepage')

/home/jonathan/anaconda3/lib/python3.6/site-packages/pandas/core/reshape/merge.py:543: UserWarning: merging between different levels can give an unintended result (2 levels on the left, 1 on the right)
  warnings.warn(msg, UserWarning)


#### Armado de la matriz de features numericos
Entiendo tiene mas sentido tomar peridos mensuales en lugar de diariamente entonces tendria n columnas de features numericos multiplicado por m donde m=cantidad de periodos tomados en este caso m=5

In [24]:
#Hago el join 
join1 = pd.merge(eventosPorFechaTrain,usersChecks, on=['person','label'],how='left')
join2 = pd.merge(usersLeads, usersConversions, on=['person','label'],how='left')
join3 = pd.merge(usersStaticPages, usersViewedProducts, on=['person','label'],how='left')
join4 = pd.merge(usersBrandListing, usersRecurrencias, on=['person','label'],how='left')
join5 = pd.merge(usersSearchProduct, usersCampaignMarketing, on=['person','label'],how='left')
join6 = pd.merge(usersSearchEngine, usersHomePage, on=['person','label'],how='left')

join12 = pd.merge(join1,join2, on=['person','label'],how='left')
join34 = pd.merge(join3,join4, on=['person','label'],how='left')
join56 = pd.merge(join5,join6, on=['person','label'],how='left')

join12n34 = pd.merge(join12,join34, on=['person','label'],how='left')
matrizDatosNumericos = pd.merge(join12n34,join56, on=['person','label'],how='left').fillna(0)


In [25]:
def change_column_order(df, col_name, index):
    cols = df.columns.tolist()
    cols.remove(col_name)
    cols.insert(index, col_name)
    return df[cols]

matrizDatosNumericos = change_column_order(matrizDatosNumericos, 'label', 62)
matrizDatosNumericos.head()

,person,Enero - cantidadEventos,Febrero - cantidadEventos,Marzo - cantidadEventos,Abril - cantidadEventos,Mayo - cantidadEventos,Enero - checkouts,Febrero - checkouts,Marzo - checkouts,Abril - checkouts,...,Febrero - searchEngine,Marzo - searchEngine,Abril - searchEngine,Mayo - searchEngine,Enero - accesos a la homepage,Febrero - accesos a la homepage,Marzo - accesos a la homepage,Abril - accesos a la homepage,Mayo - accesos a la homepage,label
0,0008ed71,0,0,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
1,000c79fe,0,0,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
2,001802e4,0,0,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
3,0019e639,1,3,0,1,9,1,3,0,1,...,3,0,1,9,1,3,0,1,9,0
4,001b0bf9,0,0,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0


In [26]:
matrizDatosNumericos.to_csv('matrizDatosNumericos.csv')

### Features categoricos

In [27]:
usersTrain['fecha'] = pd.to_datetime(usersTrain['fecha'])


In [28]:
usersTrain.columns

Index(['timestamp', 'event', 'person', 'url', 'sku', 'model', 'condition',
       'storage', 'color', 'skus', 'search_term', 'staticpage',
       'campaign_source', 'search_engine', 'channel', 'new_vs_returning',
       'city', 'region', 'country', 'device_type', 'screen_resolution',
       'operating_system_version', 'browser_version', 'label', 'fecha'],
      dtype='object')

### Encoding : Expanding Mean

In [34]:

def agruparYaplicarExpandingMean(train, evento, feature, nuevoNombre):
    dataframe = train[train['event'] == evento]\
              [['fecha','person',feature,'label']].rename(columns = {feature: nuevoNombre})\
            .sort_values(by=['fecha','person'])

    cumsum = dataframe.groupby(nuevoNombre)['label'].cumsum() - dataframe['label']
    cumcount = dataframe.groupby(nuevoNombre).cumcount()
    dataframe[nuevoNombre+'_mean_target'] = cumsum/cumcount
    dataframe.fecha = dataframe.fecha.dt.month
    dataframe = dataframe.rename(columns = {'fecha' : 'mes'}).drop(columns = nuevoNombre)
    return dataframe


In [37]:
#13mo feature: colores vistos por usuarios
usersColors = agruparYaplicarExpandingMean(usersTrain,'viewed product','color','colorViewed')

In [38]:
#14mo feature: colores comprados por el usuario por dia
usersColorsConversions = agruparYaplicarExpandingMean(usersTrain,'conversion','color','colorConversions')

In [40]:
#15mo feature: colores que llegan al checkout por usuario por dia
usersColorsCheckouts = agruparYaplicarExpandingMean(usersTrain,'checkout','color','colorCheckouts')

In [46]:
#16mo feature: condition de modelos con checkouts por usuario por dia
usersConditionCheckouts =  agruparYaplicarExpandingMean(usersTrain,'checkout','condition','conditionCheckouts')

In [47]:
#17mo feature: condition de modelos con conversiones por usuario por dia
usersConditionConversions = agruparYaplicarExpandingMean(usersTrain,'conversion','condition','conditionConversions')            

In [48]:
#18mo feature: condition de modelos vistos por usuario por dia
usersConditionViewed = agruparYaplicarExpandingMean(usersTrain,'viewed product','condition','conditionViewed') 

In [49]:
#19mo feature: modelo visto por usuario por dia 
usersModelosVistos = agruparYaplicarExpandingMean(usersTrain,'viewed product','model','modelViewed') 

In [17]:
#20mo feature: modelo con checkouts por usuario por dia
usersModelsCheckouts = agruparYaplicarExpandingMean(usersTrain,'checkout','model','modelCheckout') 

In [50]:
#21mo feature: modelo con conversiones por usuario por dia
usersModelsConversions =  agruparYaplicarExpandingMean(usersTrain,'conversion','model','modelConversion') 


In [53]:
#22mo feature: skus vistos en cierta busqueda en el site por usuario por dia
usersSkusViewedInSearchedProducts = agruparYaplicarExpandingMean(usersTrain,'searched products','skus','skusViewed')

In [54]:
#23mo feature: skus vistos en la home page por user por dia
usersSkusViewedHomePage = agruparYaplicarExpandingMean(usersTrain,'generic listing','skus','skuViewedGenericListing')

In [55]:
#24mo feature: skus vistos en un listado especifico de una marca por user por dia
usersSkusViewedBrandListing = agruparYaplicarExpandingMean(usersTrain,'brand listing','skus','skusViewedBrandListing')

In [56]:
#25mo feature: search terms por usuario por dia
usersSearchTerms = agruparYaplicarExpandingMean(usersTrain,'searched products','search_term','searchs')


In [59]:
#26mo feature: screen resolution por usuario por dia 
usersScreenResolution = agruparYaplicarExpandingMean(usersTrain,'visited site','screen_resolution','screen_resolution')

In [60]:
#27mo feature: storage de dispositivos convertidos por usuario por dia
usersStorageConversion =agruparYaplicarExpandingMean(usersTrain,'conversion','storage','storageConverted')


In [61]:
#28mo feature: campaign source por user por dia
usersCampaignSource =  agruparYaplicarExpandingMean(usersTrain,'ad campaign hit','campaign_source','campaign_source')

In [62]:
#29mo feature: country access user
usersCountry = agruparYaplicarExpandingMean(usersTrain, 'visited site','country','countryAccess')

In [27]:
#30mo feature: city access user
usersCity = agruparYaplicarExpandingMean(usersTrain,'visited site','city','cityAccess')


In [28]:
#31mo feature: region access user
usersRegionAccess = agruparYaplicarExpandingMean(usersTrain,'visited site','region','skusViewed')

In [63]:
#32mo feature: device type visited site users
usersDevicesTypes = agruparYaplicarExpandingMean(usersTrain,'visited site','device_type','device_type_Visitantes')